In [1]:
import json
import logging
from pathlib import Path
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.agents.market_agent import MarketAgent
from minference.lite.models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment
from market_agents.memory.agent_storage.agent_storage_api_utils import AgentStorageAPIUtils
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.economics.econ_agent import EconomicAgent
from market_agents.agents.personas.persona import load_persona_from_file
from market_agents.memory.document_reader import DocumentReaderFactory


In [2]:
config_path = Path("../market_agents/memory/storage_config.yaml")
config = load_config_from_yaml(str(config_path))

embedder = MemoryEmbedder(config)
storage_utils = AgentStorageAPIUtils(config=config)

hamlet_kb = MarketKnowledgeBase(
    config=config,
    table_prefix="hamlet_kb"
)
await hamlet_kb.initialize()

kb_agent = KnowledgeBaseAgent(market_kb=hamlet_kb)


INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Initializing Agent Storage API Utils with URL: http://localhost:8001
INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Initializing Agent Storage API Utils with URL: http://localhost:8001
INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Sending create tables request with payload: {'table_type': 'knowledge', 'agent_id': None, 'table_prefix': 'hamlet_kb'}


In [4]:
# Read document content
file_path = "/Users/interstellarninja/Documents/projects/MarketAgents/examples/hamlet_PDF_FolgerShakespeare.pdf"
reader = DocumentReaderFactory.get_reader(file_path)
text_content = reader.read(file_path)


In [ ]:
# Store in knowledge base with metadata
knowledge_id = await hamlet_kb.ingest_knowledge(
    text=text_content,
    metadata={
        "source": "Hamlet",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")


In [ ]:
# Test some queries
async def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = await kb_agent.retrieve(
        query=question,
        top_k=1
    )
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    await query_hamlet(query)

In [3]:
from minference.lite.models import ResponseFormat


teacher_persona = load_persona_from_file('../market_agents/agents/personas/generated_personas/Martin_Robinson.yaml')
chat_env = ChatEnvironment(name="hamlet_chat")

econ_agent = EconomicAgent(
    generate_wallet=True,
    initial_holdings={
        "ETH": 1.0,
        "USDC": 1000.0
    }
)

hamlet_agent = await MarketAgent.create(
    storage_utils=storage_utils,
    agent_id="hamlet_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024,
        response_format=ResponseFormat.tool
    ),
    knowledge_agent=kb_agent,
    environments={"chat": chat_env},
    econ_agent=econ_agent,
    persona=teacher_persona
)

# Add a test message to the chat
hamlet_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
hamlet_agent._refresh_prompts()

chat_env.mechanism.add_user_message(hamlet_agent.task)

# Generate a response
results = await hamlet_agent.run_episode(environment_name="chat")

print("Episode results:")
for step_name, result in zip(["Perception", "Action", "Reflection"], results):
    print(f"\n--- {step_name} ---")
    print(result)

INFO:market_agents.economics.econ_models:[AgentWallet] Auto-generating Ethereum wallet.
INFO:market_agents.economics.econ_models:[AgentWallet] Generated Ethereum wallet: 0x26f41421b5DA97a795F47b9b17BBEFD1012389c6
INFO:EntityRegistry:LLMConfig(daeb8542-6756-4c12-a883-4be4521816d0): Registering entity
INFO:EntityRegistry:Successfully registered LLMConfig(daeb8542-6756-4c12-a883-4be4521816d0)
INFO:EntityRegistry:LLMConfig(daeb8542-6756-4c12-a883-4be4521816d0): Successfully registered
INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Sending create tables request with payload: {'table_type': 'cognitive', 'agent_id': 'hamlet_agent', 'table_prefix': None}
INFO:market_agents.memory.agent_storage.agent_storage_api_utils:Sending create tables request with payload: {'table_type': 'episodic', 'agent_id': 'hamlet_agent', 'table_prefix': None}
INFO:EntityRegistry:LLMConfig(daeb8542-6756-4c12-a883-4be4521816d0): Registering entity
INFO:EntityRegistry:LLMConfig(daeb8542-6756-4c12-a883-4

Short-term Memories: [{'cognitive_step': 'perception', 'content': '{"monologue": "The \'To be or not to be\' soliloquy is one of the most famous passages in William Shakespeare\'s play \'Hamlet\'. In this soliloquy, Prince Hamlet reflects on the nature of existence and contemplates the idea of life and death. He questions whether it is nobler to endure the hardships of life or to take action against them, potentially ending his suffering through death. Hamlet\'s internal struggle is evident as he weighs the fear of the unknown in the afterlife against the pain and unfairness of the mortal world. This soliloquy delves into themes of existentialism, morality, and the human condition, showcasing Hamlet\'s profound philosophical introspection.", "key_observations": ["Hamlet contemplates life and death.", "He questions the nobility of enduring hardships versus taking action.", "The soliloquy reflects on existential themes.", "Hamlet fears the unknown in the afterlife."], "strategy": ["Analy

INFO:EntityRegistry:Starting parallel AI completion for 1 chat threads
INFO:EntityRegistry:Adding user message to ChatThread(4a69b778-a0e4-4c8e-bd57-07e0cf749450)
INFO:EntityRegistry:ChatMessage(77f2616c-0e42-472d-ae6d-a1660f86482f): Registering entity
INFO:EntityRegistry:Successfully registered ChatMessage(77f2616c-0e42-472d-ae6d-a1660f86482f)
INFO:EntityRegistry:ChatMessage(77f2616c-0e42-472d-ae6d-a1660f86482f): Successfully registered
INFO:EntityRegistry:Added user message(77f2616c-0e42-472d-ae6d-a1660f86482f) to ChatThread(4a69b778-a0e4-4c8e-bd57-07e0cf749450)
INFO:EntityRegistry:Preparing openai requests for 1 chat threads
INFO:EntityRegistry:Getting OpenAI request for ChatThread(4a69b778-a0e4-4c8e-bd57-07e0cf749450) with response format ResponseFormat.tool
INFO:EntityRegistry:Converting ChatThread(4a69b778-a0e4-4c8e-bd57-07e0cf749450) history to OpenAI format
INFO:EntityRegistry:Processing message: role=MessageRole.user, tool_call_id=None
INFO:EntityRegistry:Final messages for Ch

Retrieved Documents: [{'text': 'How smart a lash that speech doth give my\n conscience. The harlot’s cheek beautied with plast’ring art\n Is not more ugly to the thing that helps it\n Than is my deed to my most painted word. O heavy burden! I hear him coming. Let’s withdraw, my lord. Enter Hamlet. To be or not to be—that is the question:\n Whether ’tis nobler in the mind to suffer\n The slings and arrows of outrageous fortune,\n Or to take arms against a sea of troubles\n And, by opposing, end them. To die, to sleep—\n No more—and by a sleep to say we end\n The heartache and the thousand natural shocks\n That flesh is heir to—’tis a consummation\n Devoutly to be wished.', 'similarity': 0.8096221912357636, 'context': '...s\n well bestowed? Do you hear, let them be well used,\n for they are the abstract and brief chronicles of the\n time. After your death you were better have a bad\n epitaph than their ill report while you live.\n  My lord, I will use them according to their\n desert.\n 

INFO:EntityRegistry:Parsing results from /Users/interstellarninja/Documents/projects/MarketAgents/outputs/inference_cache/openai_results_2025-02-03_13-31-02.jsonl
INFO:EntityRegistry:Converting result for chat_thread_id: 4a69b778-a0e4-4c8e-bd57-07e0cf749450
INFO:EntityRegistry:RawOutput(aafbe449-2208-43a0-a5ce-51625726f657): Registering entity
INFO:EntityRegistry:Successfully registered RawOutput(aafbe449-2208-43a0-a5ce-51625726f657)
INFO:EntityRegistry:RawOutput(aafbe449-2208-43a0-a5ce-51625726f657): Successfully registered
INFO:EntityRegistry:GeneratedJsonObject(dcb99c54-8311-49d8-9a78-52fe63c3f5e4): Registering entity
INFO:EntityRegistry:Successfully registered GeneratedJsonObject(dcb99c54-8311-49d8-9a78-52fe63c3f5e4)
INFO:EntityRegistry:GeneratedJsonObject(dcb99c54-8311-49d8-9a78-52fe63c3f5e4): Successfully registered
INFO:EntityRegistry:Usage(b9cbf4e4-21cd-41a6-a39f-16870a826d4b): Registering entity
INFO:EntityRegistry:Successfully registered Usage(b9cbf4e4-21cd-41a6-a39f-16870a82

Episode results:

--- Perception ---
{'monologue': "The 'To be or not to be' soliloquy from Shakespeare's 'Hamlet' is a deeply philosophical reflection on life and death. It captures Hamlet's internal conflict as he contemplates the value of existence and the fear of the unknown after death. This soliloquy is significant for its exploration of existential themes, revealing Hamlet's inner turmoil and philosophical introspection.", 'key_observations': ['Hamlet contemplates life and death.', 'He questions the nobility of enduring hardships versus taking action.', 'The soliloquy reflects on existential themes.', 'Hamlet fears the unknown in the afterlife.'], 'strategy': ['Analyze the existential themes in the soliloquy.', "Consider Hamlet's internal conflict and philosophical introspection.", "Reflect on the soliloquy's impact on the play's themes."], 'confidence': 0.95}

--- Action ---
{'agent_id': 'Martin Robinson', 'action': {'thoughts': [{'reasoning': "The task requires me to explain t

In [4]:
# Print all accessible instance attributes of the hamlet_agent
for attr_name, attr_value in hamlet_agent.__dict__.items():
    value_str = str(attr_value).replace('\n', ' ')
    print(f"{attr_name}: {value_str}")

id: hamlet_agent
role: Buyer
persona: You are Martin Robinson. Martin Robinson is a Non-binary who participates in the market. They are 56 years old and Their occupation is Teacher. Their investment experience are Novice, and they have a Aggressive risk appetite.  ## Demographic Characteristics - Age: 56 - Gender: Non-binary - Education Level: Master's - Occupation: Teacher - Income Bracket: Low - Geographic Location: Urban  ## Economic Attributes - Spending Habits: Frugal - Saving Preferences: Medium - Risk Tolerance: 0.8 - Investment Experience: Novice  ## Personality Traits - Communication Style: Friendly - Decision Making Style: Emotional - Openness: 0.29 - Conscientiousness: 0.87 - Extraversion: 0.3 - Agreeableness: 0.88 - Neuroticism: 0.25  ## Hobbies and Interests Art, Sports, Gardening  ## Routines Grade assignments, Teach classes, Meet with parents  ## Skills Instruction, Communication, Classroom management  ## Dynamic Attributes - Current Mood: Neutral - Recent Life Events: S

In [5]:
from pprint import pformat
print(pformat(repr(hamlet_agent.economic_agent), indent=2, width=100))

("EconomicAgent(id='hamlet_agent', rewards=[], total_reward=0.0, "
 "wallet=AgentWallet(chain='ethereum', address='0x26f41421b5DA97a795F47b9b17BBEFD1012389c6', "
 "private_key='a3e5c48eab5400077934c90104039e88c4099f39f05f1d1a353e05ebf7a82531'), "
 "holdings=Portfolio(token_balances={'ETH': 1.0, 'USDC': 1000.0}, transaction_history=[]), "
 'generate_wallet=True)')


In [6]:
hamlet_agent.chat_thread.history

[ChatMessage(id=UUID('77f2616c-0e42-472d-ae6d-a1660f86482f'), created_at=datetime.datetime(2025, 2, 3, 18, 31, 2, 652600), timestamp=datetime.datetime(2025, 2, 3, 18, 31, 2, 652601), role=<MessageRole.user: 'user'>, content='Your are assigned with following task:\nWhat is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n\nPlease provide your output in the following JSON schema format:\nN/APerceive the current state of the chat environment:\n\n## Environment State\n- content: What is Hamlet\'s famous \'To be or not to be\' soliloquy about?\n  role: user\n  timestamp: \'2025-02-03 13:30:48\'\n\n## Recent Cognitive Memories\n- cognitive_step: perception\n  content: \'{"monologue": "The \'\'To be or not to be\'\' soliloquy is one of the most\n    famous passages in William Shakespeare\'\'s play \'\'Hamlet\'\'. In this soliloquy,\n    Prince Hamlet reflects on the nature of existence and contemplates the idea of\n    life and death. He questions whether it is nobler to endure the h